In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 데이터 불러오기 및 확인


In [48]:
import pandas as pd
import numpy as np

In [49]:
from google.colab import drive
import json
# json 파일 경로
file_path_train = '/content/drive/MyDrive/aikuthon/AIKUthon/train.json'
file_path_test = '/content/drive/MyDrive/aikuthon/AIKUthon/test.json'

# json 파일 열기 및 파싱
trainset = pd.read_json(file_path_train)
testset = pd.read_json(file_path_test)

# 데이터 확인
print(trainset)
print(testset)


                                                   info  \
0     {'caseField': '1', 'detailField': '1', 'trailF...   
1     {'caseField': '1', 'detailField': '1', 'trailF...   
2     {'caseField': '1', 'detailField': '1', 'trailF...   
3     {'caseField': '1', 'detailField': '1', 'trailF...   
4     {'caseField': '1', 'detailField': '1', 'trailF...   
...                                                 ...   
3062  {'caseField': '3', 'detailField': '5', 'trailF...   
3063  {'caseField': '3', 'detailField': '5', 'trailF...   
3064  {'caseField': '3', 'detailField': '5', 'trailF...   
3065  {'caseField': '3', 'detailField': '5', 'trailF...   
3066  {'caseField': '2', 'detailField': '6', 'trailF...   

                                          concerned  \
0                      {'acusr': '1', 'dedat': '1'}   
1     {'acusr': '1', 'dedat': '일본소속A~J,O,P주식회사, K'}   
2                {'acusr': '1', 'dedat': '국가, 자연인'}   
3                      {'acusr': '1', 'dedat': '2'}   
4               

# 데이터 전처리

In [50]:
# json 평탄화
def flatten(df):
    df = pd.json_normalize(df.to_dict(orient='records'), sep='.')
    return df

In [51]:
# concerned 전처리
# 1, 2, 3, 4를 제외한 값이 있는 경우 5로 변경하는 함수
def preprocess_concern(df):
    # 1, 2, 3, 4를 제외한 값들에 대해 5로 변경
    df['concerned.acusr'] = df['concerned.acusr'].apply(lambda x: '5' if x not in ['1', '2', '3', '4'] else x)
    df['concerned.dedat'] = df['concerned.dedat'].apply(lambda x: '5' if x not in ['1', '2', '3', '4'] else x)
    return df

In [52]:
# one hot encoding 적용
def one_hot(df):
    df = pd.get_dummies(df, columns=['info.caseField', 'info.detailField', 'info.trailField', 'concerned.acusr', 'concerned.dedat'])
    return df

In [53]:
trainset = flatten(trainset)
trainset = preprocess_concern(trainset)
trainset = one_hot(trainset)
trainset.head()

,facts,label,id,info.caseNm,info.courtNm,info.judmnAdjuDe,info.relateLaword,info.qotatPrcdnt,org.orgJdgmnCourtNm,org.orgJdgmnAdjuDe,...,concerned.acusr_1,concerned.acusr_2,concerned.acusr_3,concerned.acusr_4,concerned.acusr_5,concerned.dedat_1,concerned.dedat_2,concerned.dedat_3,concerned.dedat_4,concerned.dedat_5
0,[원고 A은 2017. 12. 31. 0000에 거주하는 친구 F을 만나러 갔다가 ...,1,0,대여금,울산지방법원,2021. 05. 25.,[],[],울산지방법원,2019. 7. 11.,...,True,False,False,False,False,True,False,False,False,False
1,[별지 청구취지 표 피징용자란에 기재된 사람들은 구 일제강점하 강제동원피해 진상규명...,1,1,손해배상(기),서울중앙지방법원,2021. 06. 07.,[구 일제강점하 강제동원피해 진상규명 등에 관한 특별법(2010. 3. 22. 법률...,[헌법재판소 2011. 8. 30. 선고 2006헌마788 결정 및 같은 날 선고 ...,,,...,True,False,False,False,False,False,False,False,False,True
2,"[2000,8. 10. 02:07 경 익산시 F 소재 G 부근에서 택시운전기사 H이...",0,2,손해배상(기),서울중앙지방법원,2021. 01. 13.,"[형사보상 및 명예회복에 관한 법률 제6조, 형사보상 및 명예회복에 관한 법률 제6조]","[대법원 2013. 12. 12. 선고 2013다201844 판결, 대법원 2012...",,,...,True,False,False,False,False,False,False,False,False,True
3,[원고는 C에 D 25.5톤 덤프트럭(이하 '이 사건 차량'이라 한다)을 지입한 후...,1,3,손해배상(기),창원지방법원,2021. 01. 28.,"[산업안전보건법 제23조, 산업안전보건기준에 관한 규칙(이하 '규칙'이라고만 한다)...","[대법원 2004. 6. 25. 선고 2004도1264 판결, 대법원 2020. 4...",창원지방법원,2019. 12. 19.,...,True,False,False,False,False,False,True,False,False,False
4,[피고는 집합건물인 0000 소재 00000(이하 '이 사건 집합건물'이라 한다)을...,1,4,부당이득금관리비,대구고등법원,2021. 04. 21.,"[집합건물법, 집합건물법 제3조 제2항 및 제3항, 집합건물법 제2조 제4호, 집합...","[대법원 2006. 6. 29. 선고 2004다3598, 3604 판결]",대구지방법원 서부지원,2019. 12. 12.,...,True,False,False,False,False,False,True,False,False,False


In [ ]:
trainset['assrs'][0]
trainset['facts'][0]

trainset['label'][0]

1

# 데이터셋 및 데이터로더 구현

In [4]:
import torch
from torch.utils.data import Dataset

class LegalDataset(Dataset):
    def __init__(self, texts, categories, labels, tokenizer, max_len):
        self.texts = texts
        self.categories = categories
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        category = self.categories[index]
        label = self.labels[index]

        # 텍스트를 토큰화
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        category_tensor = torch.tensor(category, dtype=torch.float32)
        label_tensor = torch.tensor(label, dtype=torch.float32)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'category': category_tensor,
            'label': label_tensor
        }


# 모델 구현

In [ ]:
!pip install transformers


In [ ]:
from transformers import BertTokenizer, BertModel

# BERT 토크나이저와 모델 로드
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')
model = BertModel.from_pretrained('monologg/kobert')

# 토크나이저 사용 예시
text = "안녕하세요"
inputs = tokenizer(text, return_tensors='pt')
print(inputs)

# 모델에 입력 전달 및 출력 얻기
outputs = model(**inputs)
print(outputs)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/263 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/77.8k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'KoBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


config.json:   0%|          | 0.00/426 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/369M [00:00<?, ?B/s]

{'input_ids': tensor([[2, 0, 3]]), 'token_type_ids': tensor([[0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1]])}
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.4206, -0.2654,  0.0667,  ...,  0.0047, -0.2120, -0.1067],
         [ 0.1466,  0.2745,  0.1218,  ...,  0.1801, -0.2975, -0.1744],
         [ 0.0581, -0.2740,  0.2034,  ..., -0.4992, -0.4917,  0.2777]]],
       grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 7.3357e-04, -6.6606e-02, -5.7220e-01,  7.5086e-02,  4.0251e-02,
          6.5434e-01, -6.5247e-01,  1.0613e-01,  8.8295e-02,  1.5081e-02,
          1.4897e-01, -7.3923e-02,  1.7384e-02,  1.9254e-01,  5.8342e-02,
         -3.1721e-02, -6.4720e-01, -8.6582e-02, -4.9349e-02, -8.4650e-02,
          6.9008e-02,  5.6884e-02, -6.3292e-02, -1.3818e-01, -5.2927e-02,
          5.6137e-02, -3.9697e-01,  5.5992e-03,  5.5259e-01, -5.8984e-02,
          6.4559e-01, -4.4364e-01, -6.6177e-02,  3.9578e-02, -8.5897e-01,
         -3.3367e-01, -9.5797e-03

In [ ]:
import json
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer

# 데이터셋 클래스 정의
class LegalDataset(Dataset):
    def __init__(self, file_path, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.data = self.load_data(file_path)

    def load_data(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        item = self.data[index]
        assrs = item.get('acusrAssrs', '') + ' ' + item.get('dedatAssrs', '')
        facts = item.get('facts', '')
        text = assrs + ' ' + facts

        # 텍스트를 토큰화
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        label = torch.tensor(item.get('label', 0), dtype=torch.float32)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'label': label
        }

# 하이퍼파라미터 설정
max_len = 128
batch_size = 16

# 토크나이저 초기화
tokenizer = BertTokenizer.from_pretrained('monologg/kobert')

# 데이터셋 및 데이터로더 초기화
train_dataset = LegalDataset(file_path='train.json', tokenizer=tokenizer, max_len=max_len)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# 데이터로더에서 배치 가져오기 예시
for batch in train_dataloader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    labels = batch['label']
    print(input_ids)
    print(attention_mask)
    print(labels)
    break


In [ ]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel

class LegalPredictionModel(nn.Module):
    def __init__(self, bert_model_name='monologg/kobert', num_categories=10, hidden_size=64):
        super(LegalPredictionModel, self).__init__()

        # BERT 모델 로드
        self.bert = BertModel.from_pretrained(bert_model_name)

        # 범주형 데이터 처리용 MLP
        self.category_dense = nn.Linear(num_categories, hidden_size)
        self.relu = nn.ReLU()

        # 결합된 출력 처리용 레이어
        bert_hidden_size = self.bert.config.hidden_size
        self.classifier = nn.Linear(bert_hidden_size + hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, input_ids, attention_mask, category_input):
        # BERT 처리
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output.pooler_output

        # 범주형 데이터 처리
        category_output = self.relu(self.category_dense(category_input))

        # BERT 출력과 범주형 데이터 출력 결합
        combined_output = torch.cat((pooled_output, category_output), dim=1)

        # 최종 예측
        logits = self.classifier(combined_output)
        output = self.sigmoid(logits)

        return output

# 하이퍼파라미터 설정
max_len = 128
num_categories = 10  # 범주형 데이터의 차원

# 모델 초기화
model = LegalPredictionModel(num_categories=num_categories)

# 손실 함수와 옵티마이저 설정
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# 모델 요약
print(model)